# Project | NLP Automated Customers Reviews

The goal of this project is to automate the processing of customer reviews by comparing traditional machine learning (ML) methods with a deep learning approach (transformers). Additionally, the project aims to use Generative AI to summarize reviews based on ratings and product categories and to create a visualization dashboard.